In [ ]:
!pip install -q transformers accelerate huggingface_hub


In [ ]:
import os
from huggingface_hub import login

# Set your Hugging Face API token
HF_API_TOKEN = "YOUR_HUGGINGFACE_API_TOKEN"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_API_TOKEN
login(token=HF_API_TOKEN)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "microsoft/Phi-3-mini-4k-instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Define inference pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print("✅ Model loaded and ready!")


In [ ]:
class BaseAgent:
    def __init__(self, name, prompt_template):
        self.name = name
        self.prompt_template = prompt_template

    def can_handle(self, query):
        return any(keyword in query.lower() for keyword in self.keywords)

    def handle(self, query):
        prompt = self.prompt_template.format(query=query)
        response = llm_pipeline(prompt, return_full_text=False)[0]["generated_text"]
        return f"[{self.name}]: {response.strip()}"

class MathAgent(BaseAgent):
    keywords = ["add", "subtract", "multiply", "divide", "*", "+", "-", "/"]
    def __init__(self):
        prompt = "You are a math expert. Solve the following:\nQuestion: {query}\nAnswer:"
        super().__init__("MathAgent", prompt)

class LogicAgent(BaseAgent):
    keywords = ["if", "then", "else", "logic", "true", "false"]
    def __init__(self):
        prompt = "You are a logic reasoner. Analyze:\n{query}\nResponse:"
        super().__init__("LogicAgent", prompt)

class LanguageAgent(BaseAgent):
    keywords = ["translate", "summarize", "paraphrase"]
    def __init__(self):
        prompt = "You are a language expert. Task:\n{query}\nResult:"
        super().__init__("LanguageAgent", prompt)

class RetrieverAgent(BaseAgent):
    keywords = ["who", "when", "where", "what", "which"]
    def __init__(self):
        prompt = "You are a knowledgeable assistant. Answer clearly:\n{query}\nAnswer:"
        super().__init__("RetrieverAgent", prompt)


In [ ]:
class SwarmCoordinator:
    def __init__(self, agents, recursion_limit=3):
        self.agents = agents
        self.recursion_limit = recursion_limit

    def delegate(self, query, depth=0):
        print(f"[Coordinator received]: {query}")
        for agent in self.agents:
            if agent.can_handle(query):
                response = agent.handle(query)
                print(f"{agent.name} handled it:\n")
                return response
        if depth >= self.recursion_limit:
            return "⚠️ Max recursion depth reached."
        return self.delegate("Break down: " + query, depth + 1)

# Instantiate agents and coordinator
agents = [MathAgent(), LogicAgent(), LanguageAgent(), RetrieverAgent()]
swarm = SwarmCoordinator(agents)

# Test queries
test_queries = [
    "What is the value of 7 * 6?",
    "If Alice gives Bob 5 apples, and he eats 2, what is left?",
    "Who was the first president of the United States?",
    "Translate this to French.",
    "Summarize the French Revolution.",
    "Tell me something complicated."
]

for query in test_queries:
    output = swarm.delegate(query)
    print("\nFinal Output:\n", output, "\n" + "-"*60)
